In [1]:
# tensorflow
import tensorflow as tf
import math
from model_part import conv2d
from model_part import fc
import numpy as np

In [ ]:
# Illustration
y = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32)
m = tf.constant(100.0)
c = tf.constant(10.0)

y_ = tf.multiply(m, x) + c;
l = tf.losses.mean_squared_error(labels=y, predictions=y_)

var_grad = tf.gradients(l, [y_, x])

sess = tf.Session()
l_val, var_grad_val = sess.run([l, var_grad], feed_dict={x: 1.0, y: 108})

print(l_val, var_grad_val)

In [ ]:
def loss(x, c):
    """Eigen loss."""
    x_flat = tf.reshape(x, [-1, 55*74])
    c_flat = tf.reshape(c, [-1, 55*74])

    valid_x = tf.multiply(x_flat, tf.sign(x_flat))
    
    d = tf.subtract(tf.log(valid_x), tf.log(c_flat))
    square_d = tf.square(d)
    sum_square_d = tf.reduce_sum(square_d, 1)
    sum_d = tf.reduce_sum(d, 1)
    square_sum_d = tf.square(sum_d)
    return (sum_square_d / (55.0*74.0)) - (0.5 * square_sum_d / math.pow(55*74, 2))

In [2]:
def loss(x, c):
    """Eigen loss (without log)."""
    x_flat = tf.reshape(x, [-1, 55*74])
    c_flat = tf.reshape(c, [-1, 55*74])

    valid_x = tf.multiply(x_flat, tf.sign(x_flat))
    
    d = tf.subtract(valid_x, c_flat)
    square_d = tf.square(d)
    sum_square_d = tf.reduce_sum(square_d, 1)
    sum_d = tf.reduce_sum(d, 1)
    square_sum_d = tf.square(sum_d)
    return (sum_square_d / (55.0*74.0)) - (0.5 * square_sum_d / math.pow(55*74, 2))

In [3]:
import scipy.io
mat = scipy.io.loadmat('/home/rayner/refinenet/main/exp.mat')

In [ ]:
mat['depths'].shape

In [4]:
l = loss(x=mat['logits'], c=mat['depths'])

In [ ]:
l = huber_loss(labels=mat['depths'], predictions=mat['logits'])

In [5]:
sess = tf.Session()
sess.run(l)

array([ 0.16250489])

In [6]:
x = tf.placeholder(tf.float32)
c = tf.placeholder(tf.float32)

l = loss(x, c)

var_grad = tf.gradients(l, [x])

# You can then use this symbolic gradient to compute the numerical gradient for some particular data:
sess = tf.Session()
var_grad_val = sess.run(var_grad, feed_dict={x: mat['logits'], 
                                             c: mat['depths']
                                            })

In [7]:
var_grad_val[0].shape

(55, 74)

In [8]:
import scipy.io
grads = scipy.io.loadmat('/home/rayner/refinenet/main/grads.mat')

In [9]:
grad = grads['grads']
grad.shape

(55, 74)

In [17]:
np.testing.assert_array_almost_equal(grad, var_grad_val[0],decimal=7)

AssertionError: 
Arrays are not almost equal to 7 decimals

(mismatch 100.0%)
 x: array([[ -2.3884532e-05,  -1.5770796e-04,  -1.5274121e-04, ...,
         -3.6878635e-04,   1.8065063e-04,   1.0281313e-04],
       [ -2.6669881e-04,  -2.0883813e-04,   2.5438531e-04, ...,...
 y: array([[ -2.3429635e-05,  -1.5726950e-04,  -1.5230215e-04, ...,
         -3.6837382e-04,   1.8113066e-04,   1.0328357e-04],
       [ -2.6627374e-04,  -2.0840595e-04,   2.5487438e-04, ...,...

In [11]:
np.isclose(grad, var_grad_val[0])

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [14]:
np.mean(grad)

-9.1554004204750727e-07

In [15]:
np.mean(var_grad_val[0])

-4.5782355e-07

In [16]:
np.mean(np.abs(grad - var_grad_val[0]))

4.5771700533042293e-07

In [18]:
grad

array([[ -2.38845316e-05,  -1.57707960e-04,  -1.52741206e-04, ...,
         -3.68786351e-04,   1.80650633e-04,   1.02813127e-04],
       [ -2.66698814e-04,  -2.08838134e-04,   2.54385314e-04, ...,
         -3.97353422e-04,   1.53746063e-04,   9.56172057e-05],
       [  2.97773395e-04,   1.57191410e-04,   6.35796302e-05, ...,
         -2.08946198e-06,  -2.54436364e-04,  -2.78406235e-04],
       ..., 
       [ -1.16114679e-04,  -9.26718022e-05,  -8.90399944e-05, ...,
         -3.24425390e-04,  -1.49454568e-04,  -2.69983623e-04],
       [ -1.25404172e-04,   9.28001902e-06,   1.25572812e-04, ...,
          5.24782404e-05,  -9.13597846e-05,  -3.62109184e-04],
       [  1.67997931e-04,   1.20739984e-04,  -2.47874967e-04, ...,
          2.63940821e-04,   8.73085941e-05,  -3.05469788e-05]])

In [19]:
var_grad_val[0]

array([[ -2.34296349e-05,  -1.57269504e-04,  -1.52302149e-04, ...,
         -3.68373818e-04,   1.81130657e-04,   1.03283572e-04],
       [ -2.66273739e-04,  -2.08405952e-04,   2.54874380e-04, ...,
         -3.96944437e-04,   1.54222784e-04,   9.60867837e-05],
       [  2.98267812e-04,   1.57668546e-04,   6.40452708e-05, ...,
         -1.63189486e-06,  -2.54009792e-04,  -2.77982617e-04],
       ..., 
       [ -1.15671115e-04,  -9.22253748e-05,  -8.85931076e-05, ...,
         -3.24007415e-04,  -1.49015104e-04,  -2.69558979e-04],
       [ -1.24961749e-04,   9.73898659e-06,   1.26046070e-04, ...,
          5.29425124e-05,  -9.09131923e-05,  -3.61695827e-04],
       [  1.68476399e-04,   1.21212644e-04,  -2.47447577e-04, ...,
          2.64431059e-04,   8.77771599e-05,  -3.00929078e-05]], dtype=float32)